In [1]:
import pandas as pd

# AERONEFS

In [2]:
dfa = pd.read_csv('data/aeronefs.csv')

display(dfa.shape)
display(dfa.head())

(2245, 5)

,ref_aero,type_model,debut_service,last_maint,en_maintenance
0,A330_0250,A330,2008-08-16,2023-01-08,False
1,B767_4309,B767,2011-08-01,2023-08-13,False
2,CRJ700_3824,CRJ700,2007-04-08,2023-11-29,True
3,E170_2901,E170,2012-11-20,2023-08-25,False
4,A321_1076,A321,2014-02-18,2022-12-04,False


In [13]:
unique_values = dfa['type_model'].unique()
print(unique_values)


['A330' 'B767' 'CRJ700' 'E170' 'A321' nan 'A350' 'B777' 'A340' 'B757'
 'A320' 'A380' 'E175' 'B737' 'B747' 'CRJ900']


# COMPOSANTS

In [3]:
dfc = pd.read_csv('data/composants.csv')

display(dfc.shape)
display(dfc.head())

(12257, 6)

,ref_compo,aero,desc,lifespan,taux_usure_actuel,cout_composant
0,C004,A320_2348,Éclairage d'urgence,14898,47.94,2910
1,C181,B777_2653,Système de gestion de carburant,13620,53.96,4112
2,C250,A350_5769,Dispositifs de levage (flaps),9456,63.59,3301
3,C654,A380_6627,Système de divertissement en vol,13584,73.25,7392
4,C142,B757_3104,Porte cargo,11439,2.77,4885


# DEGRADATION

In [38]:
start_date = '2023-11-29'
today = pd.Timestamp.today().strftime('%Y-%m-%d')

date_range = pd.date_range(start=start_date, end=today, freq='D')
# print(f"date_range:{date_range}")
df_each_date = []

for date in date_range:
    # print(date.strftime("%Y-%m-%d"))
    file_url = f'http://sc-e.fr/docs/degradations_{date.strftime("%Y-%m-%d")}.csv'
    
    try :
        df = pd.read_csv(file_url)
        df_each_date.append(df)
    except:
        continue

dfd = pd.concat(df_each_date)

display(dfd.shape)
display(dfd.head())

(2637, 6)

,ref_deg,linked_aero,compo_concerned,usure_cumulée,measure_day,need_replacement
0,D009477,B767_0318,C343,100.000,2023-11-29,True
1,D001145,B767_5911,C139,17.498,2023-11-29,False
2,D005001,B757_3104,C308,25.104,2023-11-29,False
3,D001380,A320_4610,C078,83.482,2023-11-29,True
4,D007247,E170_0751,C235,18.106,2023-11-29,False


# LOGS VOLS

In [5]:
dflv = pd.read_csv('data/logs_vols.csv')

display(dflv.shape)
display(dflv.head())

(32896, 6)

,ref_vol,aero_linked,jour_vol,time_en_air,sensor_data,etat_voyant
0,V07297959,A350_0001,2021-09-30 19:47:33,5.2,"{'temp': '-23°C', 'pressure': '1006 hPa', 'vib...",1
1,V03905620,NaN,2020-12-29 00:08:35,3.4,"{'temp': '12°C', 'pressure': '1006 hPa', 'vibr...",1
2,V00138907,A330_2369,2023-06-02 01:37:26,5.4,"{'temp': '-2°C', 'pressure': '1048 hPa', 'vibr...",0
3,V00821768,B757_3261,2023-02-26 23:34:08,3.0,"{'temp': '-3°C', 'pressure': '994 hPa', 'vibra...",2
4,V09664649,E170_2901,2022-09-17 14:01:56,0.5,"{'temp': '50°C', 'pressure': '1097 hPa', 'vibr...",2


test sur A320_2348

In [39]:

# Renommer les colonnes pour avoir le même nom de colonne dans chaque df
dfa_A320_2348 = dfa.rename(columns={'ref_aero': 'aero'}, inplace=True)
dfc_A320_2348 = dfc.rename(columns={'ref_compo': 'compo'}, inplace=True)
dfd_A320_2348 = dfd.rename(columns={'linked_aero': 'aero'}, inplace=True)
dfd_A320_2348 = dfd.rename(columns={'compo_concerned': 'compo'}, inplace=True)
dflv_A320_2348 = dflv.rename(columns={'aero_linked': 'aero'}, inplace=True)

# Garder les lignes qui nous intéressent dans chaque df
dfa_A320_2348 = dfa[dfa['aero'] == "A320_2348"]
dfc_A320_2348 = dfc[dfc['aero'] == "A320_2348"]
dfd_A320_2348 = dfd[dfd['aero'] == "A320_2348"]
dflv_A320_2348 = dflv[dflv['aero'] == "A320_2348"]

common_columns = ['aero','compo']
# Merge les df à partir des colonnes communes
df_A320_2348 = pd.merge(dfc_A320_2348, dfd_A320_2348, on=common_columns)
df_A320_2348 = pd.merge(df_A320_2348, dfa_A320_2348, on='aero')
df_A320_2348 = pd.merge(df_A320_2348, dflv_A320_2348, on='aero')

# Display le df merged
display(df_A320_2348)


,compo,aero,desc,lifespan,taux_usure_actuel,cout_composant,ref_deg,usure_cumulée,measure_day,need_replacement,type_model,debut_service,last_maint,en_maintenance,ref_vol,jour_vol,time_en_air,sensor_data,etat_voyant
0,C004,A320_2348,Éclairage d'urgence,14898,47.94,2910,D002933,77.736,2023-11-30,False,A320,2016-01-09,2023-08-24,False,V04127848,2022-06-25 14:15:04,3.5,"{'temp': '-46°C', 'pressure': '976 hPa', 'vibr...",1
1,C004,A320_2348,Éclairage d'urgence,14898,47.94,2910,D002933,77.736,2023-11-30,False,A320,2016-01-09,2023-08-24,False,V06367480,2020-06-16 10:26:08,9.2,"{'temp': '-42°C', 'pressure': '923 hPa', 'vibr...",1
2,C004,A320_2348,Éclairage d'urgence,14898,47.94,2910,D002933,77.736,2023-11-30,False,A320,2016-01-09,2023-08-24,False,V01279806,2022-07-26 06:55:27,2.7,"{'temp': '-43°C', 'pressure': '1030 hPa', 'vib...",0
3,C004,A320_2348,Éclairage d'urgence,14898,47.94,2910,D002933,77.736,2023-11-30,False,A320,2016-01-09,2023-08-24,False,V01278173,2023-09-22 19:23:04,9.0,"{'temp': '11°C', 'pressure': '1073 hPa', 'vibr...",0
4,C004,A320_2348,Éclairage d'urgence,14898,47.94,2910,D002933,77.736,2023-11-30,False,A320,2016-01-09,2023-08-24,False,V03079046,2023-04-07 23:42:40,5.2,"{'temp': '45°C', 'pressure': '903 hPa', 'vibra...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4768,C565,A320_2348,Système d'extinction d'incendie du moteur,13218,54.78,9976,D008418,94.434,2023-12-08,True,A320,2016-01-09,2023-08-24,False,V04636637,2021-10-28 14:24:49,7.2,"{'temp': '-45°C', 'pressure': '953 hPa', 'vibr...",1
4769,C565,A320_2348,Système d'extinction d'incendie du moteur,13218,54.78,9976,D008418,94.434,2023-12-08,True,A320,2016-01-09,2023-08-24,False,V00977609,2020-07-19 22:14:06,11.0,"{'temp': '33°C', 'pressure': '919 hPa', 'vibra...",1
4770,C565,A320_2348,Système d'extinction d'incendie du moteur,13218,54.78,9976,D008418,94.434,2023-12-08,True,A320,2016-01-09,2023-08-24,False,V02204627,2021-09-06 20:34:30,4.3,"{'temp': '8°C', 'pressure': '908 hPa', 'vibrat...",1
4771,C565,A320_2348,Système d'extinction d'incendie du moteur,13218,54.78,9976,D008418,94.434,2023-12-08,True,A320,2016-01-09,2023-08-24,False,V02497705,2020-09-05 03:59:55,0.5,"{'temp': '18°C', 'pressure': '927 hPa', 'vibra...",1
